<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/Copy_of_Antim_Bar5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###**Required Importing**

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

###**Uploading Dataset**

In [4]:
df=pd.read_csv('/content/drive/MyDrive/Content_Project/data.csv')

In [5]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


##**Preprocessing Dataset**

In [6]:
#Checking if NULL rows are present
df.isna().sum()

_id                0
title              0
datePublished      0
description        0
article           21
keywords         919
dtype: int64

In [7]:
#Removing the row with null values
df.dropna(inplace=True)

In [8]:
#Making new dataset 
df1=df[:20000]

In [9]:
df1.to_csv('data1.csv',index=False)

In [10]:
data=pd.read_csv('data1.csv')

In [11]:
data.shape

(5000, 6)

In [12]:
#Removing duplicates rows
data.drop_duplicates(inplace=True)

####**Now check the language of texts**

In [13]:
 !pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import langdetect

In [15]:
def is_english(text):
    try:
        lang = langdetect.detect(text)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

In [16]:
data['is_english'] = data['article'].apply(lambda x: is_english(x))

In [17]:
data.head()

,_id,title,datePublished,description,article,keywords,is_english
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream...",True
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct...",True
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ...",True
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par...",True
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube",True


In [18]:
data['is_english'].value_counts()

True    5000
Name: is_english, dtype: int64

In [19]:
data= data[data['is_english']==True]

###**Text Length Count**

In [20]:
from collections import Counter

In [21]:
data['text_length'] = data['article'].apply(len)

In [22]:
data.sort_values(by='text_length')

,_id,title,datePublished,description,article,keywords,is_english,text_length
1987,{'$oid': '600849a4b5425d67582e7fd5'},Mario Kart Live: Home Circuit review – TechCrunch,2020-10-15 06:05:33,Mario Kart Live: Home Circuit is a remote-cont...,\n \n \n \n \n \n Nintendo’s new RC Mario Ka...,"mario kart, Mario Kart Live: Home Circuit, Nin...",True,57
3889,{'$oid': '60084e05b5425d67582ea696'},CRISPR in the post-COVID Era – TechCrunch,2020-09-16 10:10:07,What has the global COVID-19 pandemic changed ...,What has the global COVID-19 pandemic changed ...,Biotech & Health,True,328
1538,{'$oid': '6008466fb5425d67582e6bbd'},Spycraft and Cybersecurity – TechCrunch,2020-09-17 13:00:17,The NSA is one of the world’s top spy agencies...,The NSA is one of the world’s top spy agencies...,Security & Privacy,True,329
4688,{'$oid': '60084f3bb5425d67582eb0c2'},Gillmor Gang: 3 Weeks Ago – TechCrunch,2020-08-01 11:58:28,"The Gillmor Gang — Frank Radice, Michael Markm...","The Gillmor Gang — Frank Radice, Michael Markm...","Brent Leary, Denis Pombriant, frank radice, Ke...",True,400
3666,{'$oid': '60084d88b5425d67582ea229'},Uber’s City Footprint – TechCrunch,2020-10-06 12:25:43,Uber’s operations touch upon many aspects of t...,Uber’s operations touch upon many aspects of t...,"mobility, TC Sessions: Mobility 2020",True,440
...,...,...,...,...,...,...,...,...
503,{'$oid': '60083bc6b5425d67582e2c78'},Three travel startups tell us how they’re resp...,2020-03-13 15:41:59,With the globalized world going into partial o...,With the globalized world going into partial ...,"booking platforms, coronavirus, COVID-19, Euro...",True,28279
1157,{'$oid': '60084010b5425d67582e4e73'},"Review: iPhone 12 and iPhone 12 Pro, two gems,...",2020-10-20 06:03:18,There’s an odd tension in only being able to r...,There’s an odd tension in only being able to ...,"Apple, iPhone, iphone 12, iPhone 12 Pro, review",True,32110
4872,{'$oid': '600850a2b5425d67582eb95d'},"Review: iPhone 12 and iPhone 12 Pro, two gems,...",2020-10-20 06:03:18,There’s an odd tension in only being able to r...,There’s an odd tension in only being able to ...,"Apple, iPhone, iphone 12, iPhone 12 Pro, review",True,32110
4047,{'$oid': '60084e32b5425d67582ea7fc'},Demonstrating 15 contact tracing and other too...,2020-06-05 15:47:19,"With the goal of helping the public, and state...","Personal-symptom trackers, digital contact-tra...","Apple, contact tracing, coronavirus, COVID-19,...",True,36321


####**Now consider articles with length between (300 and 30000)**

In [23]:
data= data[(data['text_length'] >= 300) & (data['text_length'] <= 20000)]

In [24]:
data.shape

(4987, 8)

##**Split the dataset into train and evaluate**

In [25]:
train_data, eval_data = train_test_split(data, test_size=0.2)

In [26]:
train_data.shape

(3989, 8)

In [27]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')

In [28]:
# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [29]:
# # Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# # Open the CSV file and read the contents
# with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
#     reader = csv.DictReader(csvfile)
#     data = [f"{tokenizer.bos_token} {row['article']} {tokenizer.eos_token}" for row in reader]

# # Write the contents to a text file
# with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
#     txtfile.write('\n'.join(data))

In [30]:
# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [31]:
# # Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# # Open the CSV file and read the contents
# with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
#     reader = csv.DictReader(csvfile)
#     data = [f"{tokenizer.bos_token} {row['article']} {tokenizer.eos_token}" for row in reader]

# # Write the contents to a text file
# with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
#     txtfile.write('\n'.join(data))

##**Load the model and tokenizer**

In [32]:
# # Load the pre-trained GPT-2 tokenizer and model
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2',pad_token='<my_pad>', mask_token='<my_mask>')
# model = GPT2LMHeadModel.from_pretrained('gpt2')

In [33]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',pad_token='<my_pad>', mask_token='<my_mask>')
tokenizer.pad_token = '<my_pad>'
tokenizer.mask_token = '<my_mask>'

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.pad_token_id
model.config.attention_mask_token_id = tokenizer.mask_token_id


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


When you add special tokens to the tokenizer vocabulary, you need to ensure that the associated word embeddings are fine-tuned or trained along with the rest of the model parameters during training. The special tokens are typically initialized with random values like the rest of the embeddings, and then updated during training based on the specific task and the data that you are using.

In the case of GPT-2, the pre-trained model already includes special tokens like <pad>, <mask>, and <unk>, as well as the start-of-sequence (`<|endoftext|>

###**Adding special tokens**

In [34]:
# # Define the special tokens
# special_tokens = {
#     "bos_token": "<BOS>",
#     "eos_token": "<EOS>",
#     "pad_token": "<PAD>",
# }

# # Add the special tokens to the tokenizer
# tokenizer.add_special_tokens(special_tokens)

##**Prepare your dataset by creating a TextDataset object**

In [35]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128,
)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [36]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


##**checkpoint using WANDB**

In [37]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import wandb
from transformers import Trainer, TrainingArguments
from transformers.integrations import WandbCallback

In [39]:
# Set up WANB
wandb.login()
wandb.init(project="my_project_tech_articles")


wandb: Currently logged in as: pathakabhishek978 (lazy_coders). Use `wandb login --relogin` to force relogin


##**Set up the trainer with the appropriate training arguments**

In [60]:


# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=0.5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    save_total_limit=2,
    #save_steps=62,
    save_strategy="epoch",
    resume_from_checkpoint='./results/checkpoint-last',
    report_to='wandb',  # Enable WandbCallback
    load_best_model_at_end=True
   
)

# Define the WandbCallback
wandb_callback = WandbCallback()

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[WandbCallback()],
)

You are adding a <class 'transformers.integrations.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback


##**Train the model**

In [61]:
torch.cuda.empty_cache()

In [62]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [49]:
trainer.evaluate()

{'eval_loss': 3.7211225032806396,
 'eval_runtime': 6.8719,
 'eval_samples_per_second': 93.424,
 'eval_steps_per_second': 11.787,
 'epoch': 2.0}

In [50]:
# # Save the best model checkpoint
trainer.save_model('./Saved_Model')


###**Now code to rehit the promt to get required length article**

In [59]:

# Set the maximum length of each segment
max_segment_length = 1024

# Set the initial prompt
prompt = "Artificial engineering is the future"

# Initialize the generated text
generated_text = prompt


# Generate segments of text and stitch them together
while len(generated_text) < 500:
    # Encode the prompt as input IDs
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate the next segment of text
    output = model.generate(input_ids=input_ids.to('cuda'),
                             max_length=max_segment_length,
                             do_sample=True,
                             top_p=0.95,
                             temperature=0.7,
                             #padding=True,
                             #truncation=True,
                             top_k=50,
                             # repetition_penalty=1.0,
                             #num_beams=1,                            
                            )
    
    # Decode the output to text
    segment = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the segment to the generated text
    generated_text += segment

    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]

# Print the generated text
print(generated_text)
final_text=generated_text


Artificial engineering is the futureArtificial engineering is the future of AI, but it has a long way to go before it is truly realized, so we are going to continue to focus on what is possible and not what is not. We are focused on this topic at present, but we have a long way to go. And we will continue to work on what is possible and not what is possible. We have a long way to go, but we have a long way to go. We will continue to work on what is possible and not what is possible. We will continue to work on what is possible and not what is possible. We are going to focus on what is possible and not what is possible. We are going to continue to work on what is possible and not what is possible. We are going to continue to work on what is possible and not what is possible. We have a long way to go, but we have a long way to go. We are going to continue to work on what is possible and not what is possible. We are going to continue to work on what is possible and not what is possible. W

In [46]:
print(len(final_text))

4503


In [47]:
type(final_text)

str

##**Use text summarization to improve article**

In [48]:
# Install required libraries
#!pip install transformers

# Import required libraries
from transformers import pipeline

# Load pre-trained summarization model
#summarizer = pipeline("summarization")
# Load pre-trained summarization model (using BART-Large-CNN)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


# Generate summary for input text
summary = summarizer(final_text, max_length=1000, min_length=30, do_sample=False)

# Print generated summary
print(summary[0]['summary_text'])


Token indices sequence length is longer than the specified maximum sequence length for this model (1028 > 1024). Running this sequence through the model will result in indexing errors


IndexError: ignored

In [ ]:
from google.colab import files
files.download('Saved_Model')


##**Below code is for save the model**

In [ ]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

In [ ]:
# trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [ ]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')

In [ ]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

In [ ]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


In [ ]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

In [ ]:
loaded_model_joblib

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')